In [1]:
import twint
import nest_asyncio

In [2]:
nest_asyncio.apply()

In [6]:
c = twint.Config()
c.Search = '#blacklivesmatter'
c.Limit = 1000
twint.run.Search(c)

1640075126438764546 2023-03-26 21:35:55 +0200 <charafe> @mac_hummer @LaJadeLo1 @JaimeLaViePaToi @Meyer_Habib @Renaissance @lesRepublicains @ECiotti @phdumont C’est exactement la même réthorique qu’utilisaient les #afrikaners pendant l’#Apartheid en #AfriqueduSud pour qualifier les noirs autochtones. Et encore la même qu’utilisaient les blancs pendant l’esclavage en #Afrique afin de justifier le traffic triangulaire. #BlackLivesMatter
1640074985065574406 2023-03-26 21:35:22 +0200 <victoria_kratel> An african refugee (who allegedly sold drugs) got killed in #Tübingen, Germany. After this tragic incident citizens are discussing possible measurements to stop migrants to deal with drugs. Well, how about this: Give refugees a real perspective! #RIP #BlackLivesMatter
1640073809700048898 2023-03-26 21:30:41 +0200 <DefenseMJJ> This brother could have been our next Malcolm X, they knew that, that's why they killed him. If Malcolm was assassinated at the age of Pac, he would be just another named

KeyboardInterrupt: 

1639764139319996419 2023-03-26 00:00:10 +0200 <RiveraSunAuthor> #RacialWealthGap: Tax the Rich  https://t.co/SBlaGv1DEL #BlackLivesMatter  #FightFor15  https://t.co/wST0Qftjkv
1639764101659529216 2023-03-26 00:00:01 +0200 <OldMainBell> Ding* ding... ding ding DING. ding Ding* #PennState #BlackLivesMatter #JusticeForAll
1639760950898348032 2023-03-25 23:47:30 +0200 <RealPureLove> If #BlacklivesMatter really matters to you. Please stop buying #ElectricVehicles , until they sort this bull$hit out. Sorry @ElonMusk, I love you Man, but this is not acceptable.
1639760309714927616 2023-03-25 23:44:57 +0200 <LucyCastilloRey> #BlackLivesMatter
1639760231268843528 2023-03-25 23:44:39 +0200 <Kaka_Laux> That simple⤵ #NoRacism🚫  #RacismoEstrutural🇧🇷  #RacismoÉCrime⚖️  #DefendTheDefenders✊🏿✊🏾✊🏽✊🏼✊🏻🏹🏳‍⚖️  #BlackLivesMatter
1639759578718978056 2023-03-25 23:42:03 +0200 <say_the_names> Miles Hall #BlackLivesMatter
1639757931704107008 2023-03-25 23:35:30 +0200 <2prithish> @DrSJaishankar Sir,  Such count